# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0011'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
       
""")
stdValue = stdVar.first()['std_length_of_stay']
print(stdValue)

12.8


In [6]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
 
      and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
             or v.vaccine_date is null) 
       
""")
stdValue_nonVac = stdVar.first()['std_length_of_stay']
print(stdValue_nonVac)

12.81


In [7]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [8]:
spark.sql(f"""

          select
              case
                    when tenant_segment like 'Community%' then "Community Hospital"
                    when tenant_segment = 'Academic' then 'Academic Hospital'
                    when tenant_segment = 'Childrens' then 'Childrens Hospital'
                    when tenant_segment = 'IDN' then 'Integrated Delivery Network'
                    when tenant_segment = 'Regional Hospital' then 'Regional Hospital'
                else 'Other'
                end as segment
              , count(h.personId) as count_of_non_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                     or v.vaccine_date is null) 
               
        group by 1
        order by 1
     
""").toPandas()

,segment,count_of_non_vaccinated_patients,mean_length_of_stay
0,Academic Hospital,17784,7.76
1,Childrens Hospital,2601,5.83
2,Community Hospital,3202,6.32
3,Integrated Delivery Network,226129,7.76
4,Other,388,5.39
5,Regional Hospital,43203,7.85


In [9]:
spark.sql(f"""

        select count(h.personId) as count_of_non_vaccinated_patients
               , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3


              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                    or v.vaccine_date is not null)  

""").toPandas()

,count_of_non_vaccinated_patients,mean_length_of_stay
0,8092,7.15


In [10]:
spark.sql(f"""

          select
                case
                    when tenant_segment like 'Community%' then "Community Hospital"
                    when tenant_segment = 'Academic' then 'Academic Hospital'
                    when tenant_segment = 'Childrens' then 'Childrens Hospital'
                    when tenant_segment = 'IDN' then 'Integrated Delivery Network'
                    when tenant_segment = 'Regional Hospital' then 'Regional Hospital'
                else 'Other'
                end as segment
              , count(h.personId) as count_of_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
        group by 1
        order by 1
     
""").toPandas()

,segment,count_of_vaccinated_patients,mean_length_of_stay
0,Academic Hospital,18,3.83
1,Childrens Hospital,2,8.00
2,Community Hospital,21,5.29
3,Integrated Delivery Network,186,5.35
4,Other,2,6.50
5,Regional Hospital,110,5.31


In [11]:
spark.sql(f"""

          select count(h.personId) as count_of_vaccinated_patients
                , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3


              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
      
""").toPandas()

,count_of_vaccinated_patients,mean_length_of_stay
0,339,5.28


In [12]:
spark.sql(f"""

          select
                case
                    when tenant_segment like 'Community%' then "Community Hospital"
                    when tenant_segment = 'Academic' then 'Academic Hospital'
                    when tenant_segment = 'Childrens' then 'Childrens Hospital'
                    when tenant_segment = 'IDN' then 'Integrated Delivery Network'
                    when tenant_segment = 'Regional Hospital' then 'Regional Hospital'
                else 'Other'
                end as segment
              , count(h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

        group by 1
        order by 1
     
""").toPandas()

,segment,count_of_all_patients,mean_length_of_stay
0,Academic Hospital,17807,7.76
1,Childrens Hospital,2603,5.83
2,Community Hospital,3229,6.31
3,Integrated Delivery Network,226360,7.76
4,Other,390,5.39
5,Regional Hospital,43326,7.85


In [13]:
spark.sql(f"""

        select count(h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3

     
""").toPandas()

,count_of_all_patients,mean_length_of_stay
0,293715,7.74
